In [4]:
import numpy as np
import matplotlib.pyplot as plt

# Parameters
num_elements = 300
num_ris_elements = 10
population_size = 100
num_generations = 100
crossover_rate = 0.8
mutation_rate = 0.1
user_distance = 100
ris_distance = 50
access_point_power = 1.0
user_noise_power = 0.01
ris_power = 0.1

# Generate positions and elements
elements = np.random.normal(0.5, 0.1, (num_elements, 3))
ris_position = np.array([0, ris_distance, 0])  # Position of RIS-UAV

# Calculate distances
access_point_distance = np.linalg.norm(elements[:, :2], axis=1)
user_distance_to_ap = np.linalg.norm(elements[:, :2] - np.array([0, user_distance]), axis=1)
user_distance_to_ris = np.linalg.norm(elements[:, :2] - ris_position[:2], axis=1)

# Channel model (simple path loss model)
def channel_model(distance):
    return 1.0 / (distance ** 2)

# SNR calculation
def calculate_snr(access_point_indices, user_indices):
    access_point_power_dB = 10 * np.log10(access_point_power)
    access_point_snr = access_point_power_dB + 10 * np.log10(channel_model(user_distance_to_ap[access_point_indices]))

    ris_power_dB = 10 * np.log10(ris_power)
    ris_snr = ris_power_dB + 10 * np.log10(channel_model(user_distance_to_ris[user_indices]))

    # Calculate the SNR improvement for each access point and RIS element pair
    combined_snr = access_point_snr[:, np.newaxis] + ris_snr
    return combined_snr


# Genetic Algorithm
def genetic_algorithm():
    population = [np.random.choice(range(num_elements), num_ris_elements, replace=False) for _ in range(population_size)]
    for _ in range(num_generations):
        fitness_values = []
        for ind in population:
            combined_snr = calculate_snr(ind, list(set(range(num_elements)) - set(ind)))
            fitness = evaluate_fitness(combined_snr)
            fitness_values.append(fitness)
        
        sorted_indices = np.argsort(fitness_values)
        sorted_indices_list = sorted_indices.tolist()  # Convert to a list of integers
        new_population = [population[i] for i in sorted_indices_list]  # Use all sorted indices

        
        crossover_population = []
        for parent1, parent2 in zip(new_population[::2], new_population[1::2]):
            if np.random.rand() < crossover_rate:
                crossover_point = np.random.randint(1, num_ris_elements)
                child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
                child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))
            else:
                child1 = parent1
                child2 = parent2
            crossover_population.extend([child1, child2])
        
        mutation_population = []
        for child in crossover_population:
            if np.random.rand() < mutation_rate:
                mutation_index = np.random.randint(num_ris_elements)
                child[mutation_index] = np.random.randint(num_elements)
            mutation_population.append(child)
        
        # population = mutation_population
        
        population = new_population
    
    best_individual = population[0]
    best_combined_snr = calculate_snr(best_individual, list(set(range(num_elements)) - set(best_individual)))
    best_energy_indices = best_individual
    best_signal_indices = list(set(range(num_elements)) - set(best_individual))
    combined_improvement_metric = evaluate_fitness(best_combined_snr)
    
    return best_energy_indices, best_signal_indices, combined_improvement_metric

# Smarter Optimization
def smarter_optimization(energy_indices, signal_indices):
    best_energy_indices = energy_indices.copy()
    best_signal_indices = signal_indices.copy()
    best_combined_improvement = 0
    for _ in range(100):
        improved = False
        for energy_index in energy_indices:
            for signal_index in signal_indices:
                new_energy_indices = [idx for idx in energy_indices if idx != energy_index]
                new_signal_indices = [idx for idx in signal_indices if idx != signal_index]
                new_energy_indices.append(signal_index)
                new_signal_indices.append(energy_index)
                combined_improvement_metric = evaluate_fitness(new_energy_indices, new_signal_indices)
                if combined_improvement_metric > best_combined_improvement:
                    best_combined_improvement = combined_improvement_metric
                    best_energy_indices = new_energy_indices
                    best_signal_indices = new_signal_indices
                    improved = True
        if not improved:
            break
    return best_energy_indices, best_signal_indices, best_combined_improvement

# Evaluate fitness
def evaluate_fitness(combined_snr):
    initial_snr = calculate_snr(range(num_ris_elements), range(num_ris_elements, num_elements))
    improved_snr = combined_snr
    snr_improvement_percentage = (abs(improved_snr - initial_snr) / initial_snr) * 100
    return snr_improvement_percentage

# Run the optimization
best_energy_indices, best_signal_indices, combined_improvement_metric = genetic_algorithm()
best_energy_indices, best_signal_indices, _ = smarter_optimization(best_energy_indices, best_signal_indices)

# Print results
print("Optimal Energy Group Indices:", best_energy_indices)
print("Optimal Signal Group Indices:", best_signal_indices)
print("Combined Improvement Metric:", combined_improvement_metric)

# Plotting
plt.figure(figsize=(8, 6))
plt.scatter(elements[best_energy_indices, 0], elements[best_energy_indices, 1], label='Energy Group')
plt.scatter(elements[best_signal_indices, 0], elements[best_signal_indices, 1], label='Signal Group')
plt.scatter(ris_position[0], ris_position[1], marker='s', color='red', label='RIS-UAV')
plt.scatter(0, user_distance, marker='x', color='green', label='User')
plt.xlabel('X Position')
plt.ylabel('Y Position')
plt.title('Optimal RIS Element Redistribution')
plt.legend()
plt.grid(True)
plt.show()


TypeError: list indices must be integers or slices, not list

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Parameters
num_elements = 300
num_energy_elements = num_elements // 2
num_signal_elements = num_elements - num_energy_elements
noise_power = 0.01

# Generate element positions
access_point = np.array([0, 0])
user = np.array([100, 0])
uav_ris = np.array([50, 50])

# Generate random channel gains (simplified example)
channel_access_user = np.random.rand(num_elements)
channel_uav_user = np.random.rand(num_elements)

def calculate_snr(channel_gain, noise_power):
    return (np.abs(channel_gain) ** 2) / noise_power

def calculate_snr_power(energy_indices, signal_indices, channel_gain, noise_power):
    # Check if the indices are valid
    if len(energy_indices) > len(channel_gain) or len(signal_indices) > len(channel_gain):
        raise ValueError("The indices are out of range")
    energy_power = np.sum(channel_gain[energy_indices])
    signal_power = np.sum(channel_gain[signal_indices])
    snr = calculate_snr(signal_power / energy_power, noise_power)
    power = energy_power + signal_power
    return snr, power

def hill_climbing_optimization(channel_gain, initial_energy_indices, initial_signal_indices, noise_power, num_iterations=100):
    current_energy_indices = initial_energy_indices.copy()
    current_signal_indices = initial_signal_indices.copy()
    # Use a try-except block to catch and handle the IndexError
    try:
        current_snr, current_power = calculate_snr_power(current_energy_indices, current_signal_indices, channel_gain, noise_power)
    except IndexError as e:
        print("Error:", e)
        return None
    
    for _ in range(num_iterations):
        improved = False
        for energy_index in current_energy_indices:
            for signal_index in current_signal_indices:
                new_energy_indices = current_energy_indices.copy()
                new_signal_indices = current_signal_indices.copy()
                new_energy_indices.remove(energy_index)
                new_signal_indices.remove(signal_index)
                new_energy_indices.append(signal_index)
                new_signal_indices.append(energy_index)
                
                # Use a try-except block to catch and handle the IndexError
                try:
                    new_snr, new_power = calculate_snr_power(new_energy_indices, new_signal_indices, channel_gain, noise_power)
                    if new_snr > current_snr:
                        current_energy_indices = new_energy_indices
                        current_signal_indices = new_signal_indices
                        current_snr = new_snr
                        current_power = new_power
                        improved = True
                except IndexError as e:
                    print("Error:", e)
                    continue
        
        if not improved:
            break
    
    return current_energy_indices, current_signal_indices, current_snr, current_power

# Run hill climbing optimization
best_energy_indices, best_signal_indices, best_snr, best_power = hill_climbing_optimization(
    np.array([channel_access_user, channel_uav_user]),
    list(range(num_energy_elements)),
    list(range(num_energy_elements, num_elements)),
    noise_power
)

# Print and plot results
print("Optimal Energy Group Indices:", best_energy_indices)
print("Optimal Signal Group Indices:", best_signal_indices)
print("Optimal SNR:", best_snr)
print("Optimal Power:", best_power)

plt.scatter(uav_ris[0], uav_ris[1], color="green", label="UAV-RIS")
plt.scatter(access_point[0], access_point[1], color="blue", label="Access Point")
plt.scatter(user[0], user[1], color="red", label="User")
plt.scatter(elements[best_energy_indices, 0], elements[best_energy_indices, 1], color="yellow", label="Energy Group")
plt.scatter(elements[best_signal_indices, 0], elements[best_signal_indices, 1], color="orange", label="Signal Group")
plt.legend()
plt.show()


ValueError: The indices are out of range

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Generate simulated data for RIS elements' characteristics (channel conditions, energy levels, signal reception)
num_elements = 300
elements = np.random.normal(0.5, 0.1, (num_elements, 4))

# Define channel gain and Rayleigh fading parameters
path_loss_exponent = 2.0
noise_power = 0.01

# Genetic Algorithm parameters
population_size = 1000
num_generations = 100
crossover_rate = 0.4
mutation_rate = 0.1

# Define the distances
user_distance = 60  # Distance between user and access point
ris_distance = 40    # Distance between RIS and access point

# Calculate initial SNR for the end user without RIS-UAV system
initial_snr_without_ris = 1 / (user_distance ** path_loss_exponent)

def calculate_snr_power(energy_indices, signal_indices):
    energy_power = np.sum(elements[energy_indices, 1])
    signal_power = np.sum(elements[signal_indices, 1])
    energy_distance = np.sum(elements[energy_indices, 2])
    signal_distance = np.sum(elements[signal_indices, 2])
    
    snr = (signal_power / (signal_distance ** 2 + 1e-10)) / (energy_power / (energy_distance ** 2 + 1e-10))
    snr *= np.random.rayleigh()
    
    power = energy_power + signal_power
    return snr, power

def evaluate_fitness(energy_indices, signal_indices):
    snr, power = calculate_snr_power(energy_indices, signal_indices)
    return 1 / (snr * power)

def genetic_algorithm():
    population = [np.random.choice(range(num_elements), num_elements // 2, replace=False) for _ in range(population_size)]
    
    for _ in range(num_generations):
        new_population = []
        for parent1, parent2 in zip(population[::2], population[1::2]):
            if np.random.rand() < crossover_rate:
                crossover_point = np.random.randint(1, num_elements // 2)
                child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
                child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))
            else:
                child1 = parent1
                child2 = parent2
            
            for child in [child1, child2]:
                if np.random.rand() < mutation_rate:
                    mutation_index = np.random.randint(num_elements // 2)
                    child[mutation_index] = np.random.randint(num_elements)
            new_population.extend([child1, child2])
        
        population = new_population
        
        best_individual = min(population, key=lambda ind: evaluate_fitness(ind, list(set(range(num_elements)) - set(ind))))
        
        best_energy_indices = best_individual
        best_signal_indices = list(set(range(num_elements)) - set(best_individual))
        best_snr, best_power = calculate_snr_power(best_energy_indices, best_signal_indices)
        
    return best_energy_indices, best_signal_indices, best_snr, best_power

def smarter_optimization(energy_indices, signal_indices):
    initial_snr, initial_power = calculate_snr_power(energy_indices, signal_indices)
    best_energy_indices = energy_indices.copy()
    best_signal_indices = signal_indices.copy()
    best_combined_improvement = 0
    
    for _ in range(100):  # Perform 100 iterations
        improved = False
        for energy_index in energy_indices:
            for signal_index in signal_indices:
                new_energy_indices = [idx for idx in energy_indices if idx != energy_index]
                new_signal_indices = [idx for idx in signal_indices if idx != signal_index]
                new_energy_indices.append(signal_index)
                new_signal_indices.append(energy_index)
                
                new_snr, new_power = calculate_snr_power(new_energy_indices, new_signal_indices)
                snr_improvement = new_snr - initial_snr
                power_reduction = initial_power - new_power
                combined_improvement_metric = snr_improvement - 0.5 * power_reduction
                
                if combined_improvement_metric > best_combined_improvement:
                    best_combined_improvement = combined_improvement_metric
                    best_energy_indices = new_energy_indices
                    best_signal_indices = new_signal_indices
                    improved = True
        
        if not improved:
            break
    
    return best_energy_indices, best_signal_indices, best_combined_improvement

# Run the Genetic Algorithm
best_energy_indices, best_signal_indices, best_snr, best_power = genetic_algorithm()

# Apply the smarter optimization strategy
best_energy_indices, best_signal_indices, combined_improvement_metric = smarter_optimization(best_energy_indices, best_signal_indices)

# Calculate initial SNR and power
initial_snr, initial_power = calculate_snr_power(range(num_elements // 2), range(num_elements // 2, num_elements))

improved_snr_with_ris = initial_snr_without_ris * best_snr

# Calculate initial SNR in dB and improved SNR in dB
initial_snr_dB = 10 * np.log10(initial_snr_without_ris)
improved_snr_dB = 10 * np.log10(improved_snr_with_ris)

# Calculate SNR improvement percentage
snr_improvement_percentage = ((improved_snr_with_ris - initial_snr_without_ris) / initial_snr_without_ris) * 100


# Print results
print("Initial SNR (Without RIS-UAV System):", initial_snr_dB, "dB")
print("Improved SNR (With RIS-UAV System):", improved_snr_dB, "dB")
print("SNR Improvement Percentage:", snr_improvement_percentage, "%")


Initial SNR (Without RIS-UAV System): -35.56302500767287 dB
Improved SNR (With RIS-UAV System): -31.9199714766839 dB
SNR Improvement Percentage: 131.36909780034668 %


In [ ]:
# # Generate simulated data for RIS elements' characteristics (channel conditions, energy levels, signal reception)
# num_elements = 300
# elements = np.random.normal(0.5, 0.1, (num_elements, 4))

# # Define channel gain and Rayleigh fading parameters
# path_loss_exponent = 2.0
# noise_power = 0.01

# # Genetic Algorithm parameters
# population_size = 100
# num_generations = 100
# crossover_rate = 0.4
# mutation_rate = 0.1

# # Define the distances
# user_distance = 60  # Distance between user and access point
# ris_distance = 40    # Distance between RIS and access point

# # Calculate initial SNR for the end user without RIS-UAV system
# initial_snr_without_ris = 1 / (user_distance ** path_loss_exponent)

# def calculate_snr_power(energy_indices, signal_indices):
#     energy_power = np.sum(elements[energy_indices, 1])
#     signal_power = np.sum(elements[signal_indices, 1])
#     energy_distance = np.sum(elements[energy_indices, 2])
#     signal_distance = np.sum(elements[signal_indices, 2])
    
#     snr = (signal_power / (signal_distance ** 2 + 1e-10)) / (energy_power / (energy_distance ** 2 + 1e-10))
#     snr *= np.random.rayleigh()
    
#     power = energy_power + signal_power
#     return snr, power

# def evaluate_fitness(individual):
#     energy_indices = [i for i in range(num_elements) if individual[i] == 0]
#     signal_indices = [i for i in range(num_elements) if individual[i] == 1]
#     snr, power = calculate_snr_power(energy_indices, signal_indices)
#     return 1 / (snr * power)

# def one_point_crossover(parent1, parent2):
#     crossover_point = np.random.randint(1, num_elements)
#     child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
#     child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))
#     return child1, child2

# def genetic_algorithm():
#     # Initialize population with random binary strings
#     population = [np.random.randint(0, 2, num_elements) for _ in range(population_size)]
    
#     for _ in range(num_generations):
#         new_population = []
#         for parent1, parent2 in zip(population[::2], population[1::2]):
#             if np.random.rand() < crossover_rate:
#                 child1, child2 = one_point_crossover(parent1, parent2)
#             else:
#                 child1 = parent1
#                 child2 = parent2
            
#             for child in [child1, child2]:
#                 if np.random.rand() < mutation_rate:
#                     mutation_index = np.random.randint(num_elements)
#                     child[mutation_index] ^= 1 # Bit-flip mutation
#             new_population.extend([child1, child2])
        
#         population = new_population
        
#         best_individual = min(population, key=evaluate_fitness)
        
#         best_energy_indices = [i for i in range(num_elements) if best_individual[i] == 0]
#         best_signal_indices = [i for i in range(num_elements) if best_individual[i] == 1]
#         best_snr, best_power = calculate_snr_power(best_energy_indices, best_signal_indices)
        
#     return best_energy_indices, best_signal_indices, best_snr, best_power

# # Run the Genetic Algorithm
# best_energy_indices, best_signal_indices, best_snr, best_power = genetic_algorithm()

# # Apply the smarter optimization strategy
# best_energy_indices, best_signal_indices, combined_improvement_metric = smarter_optimization(best_energy_indices, best_signal_indices)

# # Calculate initial SNR and power
# initial_snr, initial_power = calculate_snr_power(range(num_elements // 2), range(num_elements // 2, num_elements))

# improved_snr_with_ris = initial_snr_without_ris * best_snr

# # Calculate initial SNR in dB and improved SNR in dB
# initial_snr_dB = 10 * np.log10(initial_snr_without_ris)
# improved_snr_dB = 10 * np.log10(improved_snr_with_ris)

# # Calculate SNR improvement percentage
# snr_improvement_percentage = ((improved_snr_with_ris - initial_snr_without_ris) / initial_snr_without_ris) * 100


# # Print results
# print("Initial SNR (Without RIS-UAV System):", initial_snr_dB, "dB")
# print("Improved SNR (With RIS-UAV System):", improved_snr_dB, "dB")
# print("SNR Improvement Percentage:", snr_improvement_percentage, "%")


NameError: name 'np' is not defined